In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ctc = sc.read_h5ad('ctc_heterogeneity.h5ad')

In [3]:
ctc

AnnData object with n_obs × n_vars = 261 × 12557
    obs: 'total_counts', 'pct_counts_mt', 'pct_counts_rb', 'pct_counts_hb', 'n_genes', 'doublet_score', 'chip_id', 'patient_info', 'phage_info', '_scvi_batch', '_scvi_labels', 'leiden', 'anno', 'chip_color', 'anno_ctc'
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: '_scvi_manager_uuid', '_scvi_uuid', 'anno_colors', 'anno_ctc_colors', 'chip_id_colors', 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'umap'
    obsm: 'X_scVI', 'X_umap', '_scvi_extra_categorical_covs', '_scvi_extra_continuous_covs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [4]:
anno_mappint = {
    "0": "pEMT",
    "1": "Domancy",
    "2": "Domancy",
    "3": "Plasticity",
    "4": "Plasticity"
}

ctc.obs['anno_II'] = ctc.obs['leiden'].replace(anno_mappint)

/tmp/ipykernel_801/3516665300.py:9: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  ctc.obs['anno_II'] = ctc.obs['leiden'].replace(anno_mappint)


In [5]:
sc.tl.rank_genes_groups(ctc, groupby='anno_II', method='wilcoxon')

In [6]:
result = ctc.uns['rank_genes_groups']

# get cluster names
groups = result['names'].dtype.names

df = pd.DataFrame()

# extract top 200 genes for each group
for group in groups:
    genes = result['names'][group][:200]
    df[group] = genes

In [7]:
df

,pEMT,Domancy,Plasticity
0,PTPRM,AC090204.1,AL627171.2
1,TCF4,MTRNR2L5,PRKCB
2,NEAT1,MTRNR2L1,NRG1
3,CDH13,FP236383.1,CSMD1
4,DOCK4,MTCO3P13,EYS
...,...,...,...
195,CDH5,COL1A1,AC098588.2
196,PKP4,HBA2,DNAH14
197,ERC1,EEF1A1,RNU5B-1
198,CMIP,RNA5SP368,DLEU1


In [8]:
ctc.uns['rank_genes_groups'].keys()

dict_keys(['params', 'names', 'scores', 'pvals', 'pvals_adj', 'logfoldchanges'])

In [9]:
import gseapy as gp

for i in df.columns:
    gene_list = result['names'][i][(result['pvals_adj'][i] < 0.05) & (result['logfoldchanges'][i] > 0.25)].tolist()
    print(len(gene_list))
    enr = gp.enrichr(
        gene_list=gene_list,
        gene_sets=['GO_Biological_Process_2025', 'GO_Molecular_Function_2025', 'GO_Cellular_Component_2025'],
    )

    enr.results.to_csv(f'/mnt/c/Users/ouyangkang/Desktop/enrichment_{i}.csv')

37
126
4
